In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import statsmodels.formula.api as smf
from dowhy import CausalModel
from econml.dml import CausalForestDML

# Updated function to load and prepare data from the new CSV format
def load_and_prepare_data(data_file):
    """
    Load and prepare data from the new CSV format.

    Parameters:
    data_file: Path to CSV file containing repo metrics

    Returns:
    DataFrame ready for causal inference
    """
    # Load data
    df = pd.read_csv(data_file)

    # Convert month string to datetime
    df['date'] = pd.to_datetime(df['month'])

    # Convert HN submission date to datetime
    df['submission_date'] = pd.to_datetime(df['hn_submission_date']).dt.tz_localize(None)
    print(df[['date', 'submission_date']].dtypes)


    # Create treatment indicators
    df['hn_submitted'] = df['source'] == 'HN'

    # Create post-treatment indicator
    df['post_treatment'] = 0
    mask = (df['hn_submitted'] == 1) & (df['date'] >= df['submission_date'])
    df.loc[mask, 'post_treatment'] = 1

    # Create treatment variable
    df['treatment'] = df['hn_submitted'] * df['post_treatment']

    # Create time period column (months since start)
    min_date = df['date'].min()
    print(min_date)
    df['time_period'] = ((df['date'].dt.year - min_date.year) * 12 +
                        (df['date'].dt.month - min_date.month))

    # Rename the outcome variables for consistency with the analysis functions
    if 'new_prs' in df.columns:
        df.rename(columns={'new_prs': 'prs'}, inplace=True)
    if 'new_stars' in df.columns:
        df.rename(columns={'new_stars': 'stars'}, inplace=True)
    if 'new_forks' in df.columns:
        df.rename(columns={'new_forks': 'forks'}, inplace=True)
    if 'commit_count' in df.columns:
        df.rename(columns={'commit_count': 'commits'}, inplace=True)
    if 'active_contributors' in df.columns:
        df.rename(columns={'active_contributors': 'contributors'}, inplace=True)

    return df

# Function to match treatment and control repos based on pre-treatment trends
def match_control_repos(df, outcome_vars, n_controls_per_treated=5):
    """
    Match treated repos with control repos based on similarity in pre-treatment trends.

    Parameters:
    df: DataFrame with repo metrics
    outcome_vars: List of outcome variables to consider for matching
    n_controls_per_treated: Number of control repos to match for each treated repo

    Returns:
    DataFrame with matched treated and control repos
    """
    # Get list of treated repos
    treated_repos = df[df['hn_submitted'] == 1]['repo_url'].unique()

    # Get list of potential control repos
    control_repos = df[df['hn_submitted'] == 0]['repo_url'].unique()

    matched_controls = {}

    for treated_repo in treated_repos:
        # Get the submission date for this treated repo
        submission_date = df[(df['repo_url'] == treated_repo) &
                             (df['hn_submitted'] == 1)]['submission_date'].iloc[0]

        # Get pre-treatment data for this repo
        treated_pre = df[(df['repo_url'] == treated_repo) &
                         (df['date'] < submission_date)]

        if treated_pre.empty:
            continue  # Skip if no pre-treatment data

        # Calculate similarity scores with control repos
        similarities = []

        for control_repo in control_repos:
            # Get data for this control repo corresponding to the same time period
            control_data = df[(df['repo_url'] == control_repo) &
                             (df['date'] < submission_date)]

            if control_data.empty:
                continue

            # Calculate similarity for each outcome variable
            sim_scores = []

            for var in outcome_vars:
                # Check if we have enough data points to calculate similarity
                if len(treated_pre) >= 3 and len(control_data) >= 3:
                    # Get time series for both repos
                    treated_ts = treated_pre.sort_values('date')[var].values
                    control_ts = control_data.sort_values('date')[var].values

                    # Ensure same length by truncating the longer one
                    min_len = min(len(treated_ts), len(control_ts))
                    treated_ts = treated_ts[-min_len:]
                    control_ts = control_ts[-min_len:]

                    # Normalize the time series
                    treated_ts = (treated_ts - np.mean(treated_ts)) / (np.std(treated_ts) + 1e-10)
                    control_ts = (control_ts - np.mean(control_ts)) / (np.std(control_ts) + 1e-10)

                    # Calculate cosine similarity
                    sim = cosine_similarity([treated_ts], [control_ts])[0][0]
                    sim_scores.append(sim)
                else:
                    # If not enough data, use a neutral similarity
                    sim_scores.append(0)

            # Average similarity across all variables
            if sim_scores:
                avg_sim = np.mean(sim_scores)
                similarities.append((control_repo, avg_sim))

        # Sort by similarity (highest first) and take top n_controls_per_treated
        similarities.sort(key=lambda x: x[1], reverse=True)
        matched_controls[treated_repo] = [repo for repo, _ in similarities[:n_controls_per_treated]]

    # Create a new dataframe with treated repos and their matched controls
    matched_df = df[df['repo_url'].isin(treated_repos)].copy()

    for treated_repo, control_repos_list in matched_controls.items():
        for control_repo in control_repos_list:
            control_data = df[df['repo_url'] == control_repo].copy()
            matched_df = pd.concat([matched_df, control_data])

    return matched_df

# Function to perform staggered DiD analysis
def staggered_did_analysis(df, outcome_var):
    """
    Perform staggered Difference-in-Differences analysis.

    Parameters:
    df: Prepared DataFrame
    outcome_var: Outcome variable to measure

    Returns:
    Model results
    """
    # Create repo dummies
    repo_dummies = pd.get_dummies(df['repo_url'], prefix='repo', drop_first=True)

    # Create time period dummies
    time_dummies = pd.get_dummies(df['time_period'], prefix='time', drop_first=True)

    # Combine with original data
    model_df = pd.concat([df[['treatment', outcome_var]], repo_dummies, time_dummies], axis=1)

    # Create formula for regression
    repo_terms = ' + '.join(repo_dummies.columns)
    time_terms = ' + '.join(time_dummies.columns)
    formula = f"{outcome_var} ~ treatment + {repo_terms} + {time_terms}"

    # Run regression
    model = smf.ols(formula, data=model_df).fit(cov_type='cluster',
                                               cov_kwds={'groups': df['repo_url']})

    return model

# Perform DiD analysis using DoWhy (keeping this for compatibility)
def did_analysis(df, outcome_var='prs'):
    """
    Perform Difference-in-Differences analysis using DoWhy.

    Parameters:
    df: Prepared DataFrame
    outcome_var: Outcome variable to measure

    Returns:
    Causal effect estimate
    """
    # Define causal graph
    graph = """
    digraph {
        time_period -> %s;
        repo_url -> %s;
        treatment -> %s;
        repo_url -> treatment;
    }
    """ % (outcome_var, outcome_var, outcome_var)

    # Specify variables
    treatment_var = "treatment"
    common_causes = ["repo_url", "time_period"]

    # Create causal model
    model = CausalModel(
        data=df,
        treatment=treatment_var,
        outcome=outcome_var,
        graph=graph,
        common_causes=common_causes
    )

    # Identify causal effect
    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

    # Estimate causal effect using DiD estimator
    estimate = model.estimate_effect(
        identified_estimand,
        method_name="backdoor.econml.dml.DML",
        control_value=0,
        treatment_value=1,
        target_units="att",  # Average Treatment Effect on the Treated
        method_params={
            "init_params": {
                "model_y": LassoCV(),
                "model_t": LassoCV(),
                "model_final": RandomForestRegressor(n_estimators=100, min_samples_leaf=10),
                "fit_cate_intercept": False,
            }
        }
    )

    return estimate

# Enhanced function to check parallel trends
def check_parallel_trends(df, outcome_var='prs', plot_individual_repos=False):
    """
    Check the parallel trends assumption by plotting pre-treatment trends.

    Parameters:
    df: Prepared DataFrame
    outcome_var: Outcome variable to measure
    plot_individual_repos: Whether to plot individual repo trends or average
    """
    # Get unique repos that were submitted to HN
    hn_repos = df[df['hn_submitted'] == 1]['repo_url'].unique()

    # Create a figure
    plt.figure(figsize=(14, 8))

    # If plotting individual repos
    if plot_individual_repos and len(hn_repos) <= 20:  # Limit to 20 repos for clarity
        for repo in hn_repos:
            # Get data for this repo
            repo_data = df[df['repo_url'] == repo]

            # Get submission date
            submission_date = repo_data['submission_date'].iloc[0]
            submission_period = repo_data[
                repo_data['date'] == pd.to_datetime(submission_date).replace(day=1)
            ]['time_period'].iloc[0]

            # Plot pre-treatment trend
            pre_treatment = repo_data[repo_data['post_treatment'] == 0]
            if not pre_treatment.empty:
                plt.plot(pre_treatment['time_period'], pre_treatment[outcome_var], 'b-', alpha=0.3)

            # Plot submission point
            plt.scatter([submission_period],
                       [repo_data[repo_data['time_period'] == submission_period][outcome_var].iloc[0]],
                       color='green', marker='o', alpha=0.5)

    # Plot average trends
    # Filter for pre-treatment periods for HN repos
    pre_treatment_all = df[(df['repo_url'].isin(hn_repos)) & (df['post_treatment'] == 0)]
    control_group = df[df['hn_submitted'] == 0]

    # Aggregate by time period
    treated_pre = pre_treatment_all.groupby('time_period')[outcome_var].mean().reset_index()
    control_all = control_group.groupby('time_period')[outcome_var].mean().reset_index()

    # Plot average trends
    plt.plot(treated_pre['time_period'], treated_pre[outcome_var], 'b-',
             linewidth=3, label='Pre-treatment (HN Repos)')
    plt.plot(control_all['time_period'], control_all[outcome_var], 'r-',
             linewidth=3, label='Control (Non-HN Repos)')

    # Get the earliest treatment period
    min_treatment_period = df[df['post_treatment'] == 1]['time_period'].min()

    plt.axvline(x=min_treatment_period, color='green', linestyle='--',
                label='First Treatment')
    plt.xlabel('Time Period (Months since start)')
    plt.ylabel(f'Average {outcome_var}')
    plt.title(f'Parallel Trends Check: {outcome_var}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(f'parallel_trends_{outcome_var}.png')
    plt.close()

# Enhanced function to visualize treatment effect
def visualize_treatment_effect(df, outcome_var='prs'):
    """
    Visualize the treatment effect over time with staggered adoption.

    Parameters:
    df: Prepared DataFrame
    outcome_var: Outcome variable to measure
    """
    # Get unique repos that were submitted to HN
    hn_repos = df[df['hn_submitted'] == 1]['repo_url'].unique()

    # Create figure
    plt.figure(figsize=(16, 9))

    # Normalize time relative to treatment
    df_treated = df[df['repo_url'].isin(hn_repos)].copy()

    # For each treated repo, calculate time relative to its own treatment
    relative_time_data = []

    for repo in hn_repos:
        repo_data = df_treated[df_treated['repo_url'] == repo].copy()

        if repo_data.empty or 'submission_date' not in repo_data.columns:
            continue

        # Get the treatment time period
        treatment_date = repo_data['submission_date'].iloc[0]
        if pd.isna(treatment_date):
            continue

        treatment_period = repo_data[
            repo_data['date'] == pd.to_datetime(treatment_date).replace(day=1)
        ]['time_period'].iloc[0]

        # Calculate relative time
        repo_data['relative_time'] = repo_data['time_period'] - treatment_period

        relative_time_data.append(repo_data)

    if relative_time_data:
        relative_df = pd.concat(relative_time_data)

        # Group by relative time and calculate average
        grouped = relative_df.groupby('relative_time')[outcome_var].mean().reset_index()

        # Plot
        plt.plot(grouped['relative_time'], grouped[outcome_var], 'b-',
                linewidth=2, label='Treated Repos (Time Relative to Treatment)')

        # Add a vertical line at treatment time (relative_time = 0)
        plt.axvline(x=0, color='red', linestyle='--', label='Treatment Time')

        # Also plot the control repos average over time for comparison
        control_data = df[df['hn_submitted'] == 0].copy()
        control_avg = control_data.groupby('time_period')[outcome_var].mean().reset_index()

        # Rescale time to center around average treatment time
        avg_treatment_time = df_treated[df_treated['post_treatment'] == 1]['time_period'].min()
        control_avg['relative_time'] = control_avg['time_period'] - avg_treatment_time

        plt.plot(control_avg['relative_time'], control_avg[outcome_var], 'g-',
                linewidth=2, label='Control Repos')

        plt.xlabel('Months Relative to HN Submission')
        plt.ylabel(f'Average {outcome_var}')
        plt.title(f'Effect of HackerNews Submission on {outcome_var}')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig(f'staggered_treatment_effect_{outcome_var}.png')

    # Also create the traditional visualization
    # Aggregate data by time period and treatment status
    treatment_data = df[df['hn_submitted'] == 1].copy()
    control_data = df[df['hn_submitted'] == 0].copy()

    # For repos that were submitted to HN, mark pre and post treatment periods
    treatment_pre = treatment_data[treatment_data['post_treatment'] == 0].groupby('time_period')[outcome_var].mean().reset_index()
    treatment_post = treatment_data[treatment_data['post_treatment'] == 1].groupby('time_period')[outcome_var].mean().reset_index()
    control_all = control_data.groupby('time_period')[outcome_var].mean().reset_index()

    # Plot
    plt.figure(figsize=(14, 7))
    plt.plot(treatment_pre['time_period'], treatment_pre[outcome_var], 'b-',
             label='Pre-treatment (HN Repos)')
    plt.plot(treatment_post['time_period'], treatment_post[outcome_var], 'g-',
             label='Post-treatment (HN Repos)')
    plt.plot(control_all['time_period'], control_all[outcome_var], 'r-',
             label='Control (Non-HN Repos)')

    # Add a vertical line at the first treatment
    first_treatment = treatment_data[treatment_data['post_treatment'] == 1]['time_period'].min()
    plt.axvline(x=first_treatment, color='black', linestyle='--',
                label='First HN Submission')

    plt.xlabel('Time Period (Months since start)')
    plt.ylabel(f'Average {outcome_var}')
    plt.title(f'Effect of HackerNews Submission on {outcome_var}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(f'treatment_effect_{outcome_var}.png')
    plt.close()

# Updated main function
def main(data_file):
    """
    Main function to run the causal analysis.

    Parameters:
    data_file: Path to the data file

    Returns:
    DataFrame and results
    """
    # Define outcome variables
    outcomes = ['stars', 'forks', 'commits', 'prs', 'contributors']

    # Load and prepare data
    print("Loading repo data...")
    all_repos_data = load_and_prepare_data(data_file)

    # Check if we have treatment and control repos
    n_treated = all_repos_data[all_repos_data['hn_submitted'] == 1]['repo_url'].nunique()
    n_control = all_repos_data[all_repos_data['hn_submitted'] == 0]['repo_url'].nunique()

    print(f"Found {n_treated} treated repos and {n_control} control repos")

    # If we don't have control repos, we need to get some from an external source
    if n_control == 0:
        print("No control repos found. Please provide control repos in the dataset.")
        return None, None

    # Match treatment and control repos
    print("Matching treatment and control repos...")
    matched_data = match_control_repos(all_repos_data, outcomes)

    # Store results
    results = {}

    for outcome_var in outcomes:
        print(f"\nAnalyzing causal effect on {outcome_var}...")

        # Drop NaN values for this specific outcome
        analysis_data = matched_data.dropna(subset=[outcome_var])

        # Check parallel trends assumption
        print("Checking parallel trends assumption...")
        check_parallel_trends(analysis_data, outcome_var)

        # Run staggered DiD analysis
        print("Running staggered DiD analysis...")
        model = staggered_did_analysis(analysis_data, outcome_var)

        # Calculate DiD estimate (the coefficient of the treatment variable)
        did_estimate = model.params['treatment']
        did_stderr = model.bse['treatment']
        p_value = model.pvalues['treatment']

        print(f"Estimated causal effect on {outcome_var}: {did_estimate:.4f}")
        print(f"Standard error: {did_stderr:.4f}")
        print(f"p-value: {p_value:.4f}")

        # Also run the DML estimator from DoWhy for comparison
        try:
            dowhy_estimate = did_analysis(analysis_data, outcome_var)
            print(f"DoWhy estimated causal effect: {dowhy_estimate.value:.4f}")
        except Exception as e:
            print(f"DoWhy analysis failed: {str(e)}")
            dowhy_estimate = None

        # Visualize treatment effect
        print("Visualizing treatment effect...")
        visualize_treatment_effect(analysis_data, outcome_var)

        # Store results
        results[outcome_var] = {
            'estimate': did_estimate,
            'stderr': did_stderr,
            'p_value': p_value,
            'dowhy_estimate': dowhy_estimate.value if dowhy_estimate else None
        }

        # Optional: Analyze heterogeneous effects
        print("Analyzing heterogeneous effects...")
        try:
            cf_model, df_with_cate = heterogeneous_effects(analysis_data, outcome_var)

            # Save the top and bottom repos by treatment effect
            top_repos = df_with_cate.sort_values('cate', ascending=False).head(10)
            bottom_repos = df_with_cate.sort_values('cate', ascending=True).head(10)

            print(f"Top 10 repos with highest treatment effect on {outcome_var}:")
            print(top_repos[['repo_url', 'cate']].drop_duplicates('repo_url').head())

            print(f"Bottom 10 repos with lowest treatment effect on {outcome_var}:")
            print(bottom_repos[['repo_url', 'cate']].drop_duplicates('repo_url').head())

        except Exception as e:
            print(f"Heterogeneous effects analysis failed: {str(e)}")

    # Summary of results
    print("\nSummary of Results:")
    print("-" * 50)
    for outcome, result in results.items():
        print(f"Outcome: {outcome}")
        print(f"  Causal Effect: {result['estimate']:.4f} ± {result['stderr']:.4f}")
        print(f"  p-value: {result['p_value']:.4f}")
        if result['p_value'] < 0.05:
            print("  Result is statistically significant at p < 0.05")
        else:
            print("  Result is not statistically significant at p < 0.05")
        print("-" * 50)

    return matched_data, results

# Call this function with your data file path
# matched_data, results = main("your_data_file.csv")

# Function to select optimal control repos by examining pre-treatment trends
def select_optimal_controls(df, outcome_vars, max_controls=500):
    """
    Select optimal control repos that best satisfy the parallel trends assumption.

    Parameters:
    df: DataFrame with repo metrics
    outcome_vars: List of outcome variables to consider
    max_controls: Maximum number of control repos to select

    Returns:
    List of optimal control repo URLs
    """
    # Get all treated repos
    treated_repos = df[df['hn_submitted'] == 1]['repo_url'].unique()

    # Get all potential control repos
    potential_controls = df[df['hn_submitted'] == 0]['repo_url'].unique()

    # If we have no potential controls, return an empty list
    if len(potential_controls) == 0:
        return []

    # For each time period, calculate the average of each outcome variable for treated repos
    treated_avgs = {}
    for var in outcome_vars:
        treated_avgs[var] = df[df['hn_submitted'] == 1].groupby('time_period')[var].mean()

    # Calculate how well each potential control repo's pre-treatment trend matches the treated repos
    control_scores = []

    for control_repo in potential_controls:
        control_data = df[df['repo_url'] == control_repo]

        score = 0
        for var in outcome_vars:
            # Get the control repo's values
            control_values = control_data.set_index('time_period')[var]

            # Calculate score based on how well it matches the treated repos' average
            # (only for time periods where both have data)
            common_periods = treated_avgs[var].index.intersection(control_values.index)

            if len(common_periods) > 2:  # Need at least 3 points to evaluate trend
                treated_vals = treated_avgs[var].loc[common_periods].values
                control_vals = control_values.loc[common_periods].values

                # Normalize to account for scale differences
                treated_norm = (treated_vals - np.mean(treated_vals)) / (np.std(treated_vals) + 1e-10)
                control_norm = (control_vals - np.mean(control_vals)) / (np.std(control_vals) + 1e-10)

                # Calculate correlation (how well the trends match)
                if np.std(treated_norm) > 0 and np.std(control_norm) > 0:
                    corr = np.corrcoef(treated_norm, control_norm)[0, 1]
                    score += corr

        # Average the score across all outcome variables
        if len(outcome_vars) > 0:
            score /= len(outcome_vars)
            control_scores.append((control_repo, score))

    # Sort by score (higher is better)
    control_scores.sort(key=lambda x: x[1], reverse=True)

    # Return the top control repos
    return [repo for repo, _ in control_scores[:max_controls]]

# Function to heterogeneous treatment effects - keeping this from original
def heterogeneous_effects(df, outcome_var='prs'):
    """
    Analyze heterogeneous treatment effects using CausalForest.

    Parameters:
    df: Prepared DataFrame
    outcome_var: Outcome variable to measure

    Returns:
    CausalForest model and DataFrame with CATE estimates
    """
    # Select numeric features (excluding the outcome and treatment variables)
    feature_cols = [col for col in df.columns if df[col].dtype in [np.int64, np.float64]]
    feature_cols = [col for col in feature_cols if col not in
                    [outcome_var, 'treatment', 'hn_submitted', 'post_treatment']]

    # Prepare features
    features = df[feature_cols]
    treatment = df['treatment']
    outcome = df[outcome_var]

    # Drop rows with NaN values
    mask = ~(features.isna().any(axis=1) | treatment.isna() | outcome.isna())
    features = features[mask]
    treatment = treatment[mask]
    outcome = outcome[mask]

    # Convert to numpy arrays and ensure correct shape
    features = features.values
    treatment = treatment.values.ravel()  # Use ravel() to ensure 1D array
    outcome = outcome.values.ravel()      # Use ravel() to ensure 1D array

    # Fit causal forest model
    cf_model = CausalForestDML(
        model_y=LassoCV(),  # Will use default models
        model_t=LassoCV(),
        n_estimators=100,
        min_samples_leaf=10,
        fit_cate_intercept=False,
    )
    cf_model.fit(features, treatment, outcome)

    # Calculate conditional treatment effects
    cate_estimates = cf_model.effect(features)

    # Add CATE estimates to DataFrame
    df_with_cate = df[mask].copy()
    df_with_cate['cate'] = cate_estimates

    return cf_model, df_with_cate

In [28]:
# Load your dataset
matched_data, results = main("./hn-stories-gh-ai-metrics.csv")

# Review the results
for outcome, result in results.items():
    print(f"Outcome: {outcome}")
    print(f"  Causal Effect: {result['estimate']:.4f} ± {result['stderr']:.4f}")
    print(f"  p-value: {result['p_value']:.4f}")

Loading repo data...
date               datetime64[ns]
submission_date    datetime64[ns]
dtype: object
2022-05-01 00:00:00
Found 1814 treated repos and 0 control repos
No control repos found. Please provide control repos in the dataset.


AttributeError: 'NoneType' object has no attribute 'items'